<a href="https://colab.research.google.com/github/vieri2006/machine-learning-course/blob/master/Machine_Learning_Hand_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to my assignment
## Vieri Wijaya - Dicoding

### Fitur yang tidak diajarkan:
1. Menggunakan dropout untuk mengurangi overfitting
2. Menggunakan activation function bernama `LeakyReLu` yang konon katanya lebih efisien
3. Melakukan train-test split karena jika tidak, ada dataset train yang masuk ke dataset validasi sehingga mengurangi keabsahan validasi

## Import Libs

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#to split dataset
import shutil
import os
import math
import zipfile
from google.colab import files

#to plot and upload results
import numpy as np
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

Using TensorFlow backend.


## Download Datasets dan prepare datasets dirs

In [ ]:
!wget --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip \
  -O /tmp/rockpaperscissors.zip

--2020-05-08 19:39:16--  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip
Resolving dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)... 52.239.197.36
Connecting to dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)|52.239.197.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322873683 (308M) [application/zip]
Saving to: ‘/tmp/rockpaperscissors.zip’

/tmp/rockpapersciss 100%[===================>] 307.92M  7.98MB/s    in 43s     

2020-05-08 19:40:01 (7.08 MB/s) - ‘/tmp/rockpaperscissors.zip’ saved [322873683/322873683]



In [ ]:
srcdir = '/tmp/data_awal/'
workdir = '/tmp/dataset/'

local_zip = '/tmp/rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall(srcdir)
zip_ref.close()

srcdir += 'rockpaperscissors/'
shutil.rmtree(srcdir+'/rps-cv-images')

#Jika merun ulang, biar tidak ada duplikat
try:
  shutil.rmtree(workdir)
  os.mkdir(workdir)
except:
  os.mkdir(workdir)

In [ ]:
os.chdir(srcdir)
category_list = list(filter(lambda x: os.path.isdir(x), os.listdir()))

for category in category_list:
  print(category)

data_set_dirs= ['train', 'valid']
for dsdirs in data_set_dirs:
  path = workdir + '/'+ dsdirs
  os.mkdir(path)

train_prop = 0.85
valid_prop = 1 - train_prop

for cat in category_list: 
  src_path = srcdir + '/' + cat
  dest_dir1 = workdir + '/train/' + cat
  dest_dir2 = workdir + '/valid/' + cat
  dest_dirs_list = [dest_dir1, dest_dir2]

  os.chdir(src_path)
  files_ = [f for f in os.listdir() if os.path.isfile(f)]
  train_count = math.ceil(train_prop * len(files_))
  train_data_list = files_[0 : train_count]
  valid_data_list = files_[train_count :] 

  for dirs in dest_dirs_list:
    os.mkdir(dirs)

  for train_data in train_data_list:
    train_path = src_path + '/' + train_data
    shutil.copy(train_path, dest_dir1)

  for valid_data in valid_data_list:
    valid_path = src_path + '/' + valid_data
    shutil.copy(valid_path, dest_dir2)

train_dir = os.path.join(workdir, 'train')
valid_dir = os.path.join(workdir, 'valid')

paper
scissors
rock


## Image Generator

In [ ]:
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    zoom_range = 0.15,
                    shear_range = 0.2,
                    fill_mode = 'nearest')
 
test_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')

In [ ]:
train_generator = train_datagen.flow_from_directory(
        train_dir, 
        target_size=(150, 150), 
        batch_size=32,
        #color_mode='grayscale',
        class_mode='categorical')
 
validation_generator = test_datagen.flow_from_directory(
        valid_dir, 
        #color_mode='grayscale',
        target_size=(150, 150),
        batch_size=32, 
        class_mode='categorical')

Found 1862 images belonging to 3 classes.
Found 326 images belonging to 3 classes.


## CNN Model

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), input_shape=(150, 150, 3)),
    tf.keras.layers.LeakyReLU(0.2),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3)),
    tf.keras.layers.LeakyReLU(0.2),
    tf.keras.layers.Conv2D(64, (3,3)),
    tf.keras.layers.LeakyReLU(0.2),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3)),
    tf.keras.layers.LeakyReLU(0.2),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(256, (3,3)),
    tf.keras.layers.LeakyReLU(0.2),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(150, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 72, 72, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 70, 70, 64)        36928     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 70, 70, 64)        0

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.optimizers.Adam(),
              metrics=['accuracy'])

## CNN Training

In [ ]:
model.fit(
      train_generator,
      steps_per_epoch=25, 
      epochs=10,
      validation_data=validation_generator,
      validation_steps=10, 
      )

Epoch 1/10
25/25 [==============================] - 71s 3s/step - loss: 0.9047 - accuracy: 0.5475 - val_loss: 0.5787 - val_accuracy: 0.8156
Epoch 2/10
25/25 [==============================] - 67s 3s/step - loss: 0.4806 - accuracy: 0.8175 - val_loss: 0.2820 - val_accuracy: 0.9031
Epoch 3/10
25/25 [==============================] - 65s 3s/step - loss: 0.3591 - accuracy: 0.8734 - val_loss: 0.2999 - val_accuracy: 0.8969
Epoch 4/10
25/25 [==============================] - 67s 3s/step - loss: 0.2898 - accuracy: 0.8900 - val_loss: 0.1780 - val_accuracy: 0.9375
Epoch 5/10
25/25 [==============================] - 65s 3s/step - loss: 0.2574 - accuracy: 0.9005 - val_loss: 0.2480 - val_accuracy: 0.9094
Epoch 6/10
25/25 [==============================] - 67s 3s/step - loss: 0.1790 - accuracy: 0.9438 - val_loss: 0.1341 - val_accuracy: 0.9500
Epoch 7/10
25/25 [==============================] - 67s 3s/step - loss: 0.1577 - accuracy: 0.9438 - val_loss: 0.1125 - val_accuracy: 0.9594
Epoch 8/10
25/25 [==

## Production

In [ ]:
uploaded = files.upload()
 
for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150,150), #color_mode='grayscale'
                       )
  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
 
  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)[0]
  
  print(fn)
  
  if classes[0]:
    print('Paper')
  elif classes[1]:
    print('Rock')
  else:
    print('Scissors')
